In [330]:
import os
import openai
from gitdb.util import write
from sympy import content
!pip install gitpython


In [331]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [332]:
from git import Repo

In [333]:
from pathlib import Path

In [334]:
pwd

'H:\\02_AIWork\\GitHub\\Data'

In [335]:
PATH_TO_BLOG = Path("H:\\02_AIWork\\GitHub\\Data")

In [336]:
PATH_TO_BLOG = PATH_TO_BLOG_REPO.parent

In [337]:
PATH_TO_CONTENT = PATH_TO_BLOG/"content"

In [338]:
PATH_TO_CONTENT

WindowsPath('H:/02_AIWork/GitHub/Data/content')

In [339]:
PATH_TO_CONTENT.mkdir(exist_ok=True, parents=True)

In [340]:
def update_blog(commit_message='Updates blog'):
    # Открываем репозиторий Git
    repo = Repo(PATH_TO_BLOG)
    # git add
    repo.git.add(A=True) # или repo.git.add(".")
    # Создаем коммит
    repo.index.commit(commit_message)
    # Отправляем изменения в удаленный репозиторий
    origin = repo.remote(name='origin')
    origin.push()

In [341]:
# random_text_string = "jakdfdjfkljasfljdjflsdajf"

In [342]:
# with open(PATH_TO_BLOG/"index.html", "w") as f:
#    f.write(random_text_string)

In [343]:
update_blog()

In [344]:
import shutil
def create_new_blog(title,content,cover_image):
    cover_image = Path(cover_image)
    
    # Проверяем, есть ли файлы .html
    existing_files = list(PATH_TO_CONTENT.glob("*.html"))
    
    files = len(existing_files)  # Количество существующих блогов
    new_title = f"{files+1}.html"
    path_to_new_content = PATH_TO_CONTENT / new_title
    
    shutil.copy(cover_image, PATH_TO_CONTENT)
    
    if not os.path.exists(path_to_new_content):
        # WHITE A NEW HTML FILE
        with open(path_to_new_content, "w") as f:
            f.write("<!DOCTYPE html>\n")
            f.write("<html>\n")
            f.write("<head>\n")
            f.write("<title>" + title + "</title>\n")
            f.write("</head>\n")
            
            f.write("<body>\n")
            f.write(f"<img src='{cover_image.name}' alt='Cover Image'> <br />\n")
            f.write(f"<h1> {title} </h1>\n")
            # OpenAI --> Completion GPT --> "hello\nblog post\n"
            f.write(content.replace("\n", "<br />\n"))
            f.write("</body>\n")
            f.write("</html>\n")
            print("Blog Created")
            return path_to_new_content
            
    else:
        raise FileNotFoundError("Файл уже существует, пожалуйста, проверьте еще раз свое имя! Отмена!")
    

In [356]:
path_to_new_content = create_new_blog('Test_title','skdajfldjslkjdsf','example_download.png')

Blog Created


In [359]:
#index.html ---> Blog post

In [360]:
from bs4 import BeautifulSoup as Soup

In [361]:
# pip install BeautifulSoup4

In [362]:
with open(PATH_TO_BLOG/"index.html") as index:
    soup = Soup(index.read())

In [363]:
str(soup)

'<html lang="en">\n<head>\n<meta charset="utf-8"/>\n<meta content="width=device-width, initial-scale=1" name="viewport"/>\n<title>\n   Document\n  </title>\n</head>\n<body>\n<h1> My Blog Home Page\n  </h1>\n<a href="index.html">\n   Home\n  </a>\n<a href="content\\8.html">\n   8\n  </a>\n</body>\n</html>\n'

In [364]:
# checking duplicate links

# write blog post link ---> index.html

In [365]:
def check_for_duplicate_links(path_to_new_content, links):
    urls = [str(link.get("href")) for link in links]  
    content_path = str(Path(*path_to_new_content.parts[-2:]))
    return content_path in urls


In [366]:
def write_to_index(path_to_new_content):
    with open(PATH_TO_BLOG/'index.html') as index:
        soup = Soup(index.read())
        
    links = soup.find_all("a")    
    
    # Проверяем, есть ли ссылки в файле index.html
    if not links:
        print("⚠ Внимание: В index.html нет ссылок! Добавляем первую ссылку.")
        with open(PATH_TO_BLOG/'index.html', 'w') as f:
            f.write(f'<a href="{Path(*path_to_new_content.parts[-2:])}">{path_to_new_content.name.split(".")[0]}</a>\n')
        return
    
    last_link = links[-1]  # Теперь эта строка безопасна

    if check_for_duplicate_links(path_to_new_content, links):
        raise ValueError("❌ Ошибка: Ссылка уже существует!")

    # Создаем новую ссылку
    links_to_new_blog = soup.new_tag("a", href=Path(*path_to_new_content.parts[-2:]))
    links_to_new_blog.string = path_to_new_content.name.split('.')[0]

    last_link.insert_after(links_to_new_blog)  # Вставляем новую ссылку

    # Записываем обновленный index.html
    with open(PATH_TO_BLOG/'index.html', 'w') as f:
        f.write(str(soup.prettify(formatter="html")))

    print("✅ Ссылка успешно добавлена в index.html!")


In [367]:
write_to_index(path_to_new_content)

✅ Ссылка успешно добавлена в index.html!


In [329]:
update_blog()